# Load Packages

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import VGG16, Xception, InceptionV3, MobileNet, ResNet50
from tensorflow.keras.preprocessing import image

# Evaluation Functions

In [ ]:
def plot_predictions_for_class(data, class_id, figsize=(10,7)):
    """
    Plots the distribution of the predictions given a label
    """
    subset = data[data["Labels"] == class_id]
    plt.figure(figsize=figsize)
    plt.title("Count per Predicted Label")
    plt.xlabel("Food Item")
    plt.ylabel("Count")
    value_counts = subset["Predictions"].value_counts().plot(kind="bar")
    return value_counts

def get_most_accurate(data, k=1):
    """
    Returns top k most accurate predictions
    """
    subset = data[data["Labels"]==data["Predictions"]]
    results = (subset["Labels"].value_counts()/data["Labels"].value_counts()).sort_values(ascending=False)[:k]
    return results

# Create Image Generators

In [ ]:
batch_size = 32
shape = (200, 200)

# data augmentation
datagen = ImageDataGenerator(
    rescale=1. / 255,
    horizontal_flip=True, # randomly flip images
    width_shift_range=0.1, # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2, # randomly shift images vertically (fraction of total height)
    rotation_range=10, # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.05, # randomly zoom image
    brightness_range=[0.4, 0.8],
    fill_mode="reflect"
    ) 

train_generator = datagen.flow_from_directory(
    directory="../images/train",
    target_size=shape,
    color_mode="rgb",
    shuffle=True,
    batch_size=batch_size,
    class_mode="categorical",
    seed=2019)

valid_generator = datagen.flow_from_directory(
    directory="../images/valid",
    target_size=shape,
    color_mode="rgb",
    shuffle=True,
    batch_size=batch_size,
    class_mode="categorical",
    seed=2019)

test_generator = datagen.flow_from_directory(
    directory="../images/test",
    target_size=shape,
    color_mode="rgb",
    shuffle=False,
    batch_size=1,
    class_mode="categorical")

num_classes = len(train_generator.class_indices)

# create step size
STEP_SIZE_TRAIN=np.ceil(train_generator.n/train_generator.batch_size)
STEP_SIZE_VALID=np.ceil(valid_generator.n/valid_generator.batch_size)
STEP_SIZE_TEST=np.ceil(test_generator.n/test_generator.batch_size)

# Create Base Model

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(shape[0], shape[1], 3), padding='same', use_bias=False, kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), input_shape=(shape[0], shape[1], 3), padding='same', use_bias=False, kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3, 3), padding='same', use_bias=False, kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', use_bias=False, kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128, (3, 3), padding='same', use_bias=False, kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation="softmax"))

In [ ]:
# compile
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.0001, decay=1e-6),
              metrics=['accuracy'])

In [ ]:
# callbacks
earlyStopping = EarlyStopping(monitor="val_loss", patience=10, verbose=0, mode="min")
checkpoint = ModelCheckpoint(os.path.join("..", "models", "model-{epoch:03d}-{acc:03f}-{val_acc:03f}.h5"), verbose=1, 
                             monitor="val_loss", save_best_only=True, mode="auto")
reduce_lr_loss = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode="min")

In [ ]:
# fit base model
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    callbacks=[earlyStopping, checkpoint, reduce_lr_loss],
                    epochs=100, verbose=2)

# Transfer Learning with MobileNet

In [ ]:
# load base MobileNet
base_mn = MobileNet(weights='imagenet', include_top=False, input_shape=(shape[0], shape[1], 3))

In [ ]:
# add last few layers
top_block = base_mn.output

top_block = GlobalAveragePooling2D()(top_block) # pool over height/width to reduce number of parameters
top_block = Dense(256, activation='relu')(top_block) # add a Dense layer
predictions = Dense(num_classes, activation='softmax')(top_block) # add another Dense layer
mn_transfer = Model(inputs=base_mn.input, outputs=predictions)

In [ ]:
# unfreeze last few layers
for i, layer in enumerate(reversed(mn_transfer.layers)):
    layer.trainable = True
    if i > 15:
        break

In [ ]:
# compile model
mn_transfer.compile(loss="categorical_crossentropy",
              optimizer=RMSprop(lr=0.0002),
              metrics=["accuracy"])

In [ ]:
# fit model
checkpoint3 = ModelCheckpoint(os.path.join("..","keras_models", "model-mobilenet-RMSprop0.0002-{epoch:03d}-{acc:03f}-{val_acc:03f}.h5"), 
                              verbose=1, monitor="val_loss", save_best_only=True, mode="auto")
history_mn = mn_transfer.fit_generator(generator=train_generator,
                                       steps_per_epoch=STEP_SIZE_TRAIN,
                                       validation_data=valid_generator,
                                       validation_steps=STEP_SIZE_VALID,
                                       callbacks=[earlyStopping, checkpoint3],
                                       epochs=10, verbose=1)

## Fine Tune

In [ ]:
# load model
transfer = load_model(os.path.join("..", "keras_models", "model-mobilenet-RMSprop0.0002-002-0.509663-0.484030.h5"))

In [ ]:
# compile
transfer.compile(loss="categorical_crossentropy",
              optimizer=RMSprop(lr=0.0001), 
              metrics=["accuracy"])

In [ ]:
# continue fitting
checkpoint3 = ModelCheckpoint(os.path.join("..", "keras_models", "model-mobilenet-RMSprop0.0002to0.0001-{epoch:03d}-{acc:03f}-{val_acc:03f}.h5"), 
                              verbose=1, monitor="val_loss", save_best_only=True, mode="auto")
history_mn = transfer.fit_generator(generator=train_generator,
                                       steps_per_epoch=STEP_SIZE_TRAIN,
                                       validation_data=valid_generator,
                                       validation_steps=STEP_SIZE_VALID,
                                       callbacks=[earlyStopping, checkpoint3],
                                       epochs=100, verbose=1, initial_epoch=2)

In [ ]:
# evaluate
val_loss, val_acc = transfer.evaluate_generator(generator=valid_generator, steps=STEP_SIZE_VALID, verbose=1)
print("Val Loss: {} \nVal Accuracy: {}".format(val_loss, val_acc))

## Predict Tests

In [ ]:
# predict test images
test_generator.reset()
pred = transfer.predict_generator(test_generator, steps=STEP_SIZE_TEST, verbose=1)

In [ ]:
# clean predictions
predictions = pred.argmax(axis=-1)
labels = (test_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predicted_labels = [labels[k] for k in predictions]

In [ ]:
# create prediction dataframe
filenames = test_generator.filenames
correct_labels = [filename[:filename.find("\\")] for filename in filenames]
results = pd.DataFrame({"Filename": filenames, "Labels": correct_labels, "Predictions": predicted_labels})

In [ ]:
get_most_accurate(results, 10)

In [ ]:
plot_predictions_for_class(results, "Kuromame")

# Example Predictions

In [ ]:
# predict 1 image
source = "valid"
random_folder = np.random.choice(os.listdir(os.path.join("..", "images", source)))
random_image = np.random.choice(os.listdir(os.path.join("..", "images", source, random_folder)))
img = image.load_img(os.path.join("..", "images", source, random_folder, random_image), target_size = (shape[0], shape[1]))
plt.imshow(img)
img = image.img_to_array(img) / 255
img = np.expand_dims(img, axis = 0)

print("Actual:", random_image)
print("Predicted:", labels[best_model.predict(img).argmax(axis=-1)[0]])